In [1]:
import numpy as np
import pandas as pd
import time
from time import time
import pickle
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
from multiprocessing import Pool
from joblib import Parallel, delayed

In [2]:
# Load data from a CSV file
nf = pd.read_csv(r'C:\Users\smahm\NFSIM\C35\MTD40B50.csv')

# Rename the columns for clarity
nf.columns = np.append(np.array([-1]), np.arange(0, 135, 15))
nf = nf.reset_index(drop=True)  # Reset the index and drop the old index

In [3]:
# Activation probabilities for different experiments (Non-Flavopiridol)
M2 = [0.13, 0.22, 0.32, 0.41, 0.46, 0.51]  # March Experiment
S2 = [0.32, 0.40, 0.48, 0.54, 0.59, 0.64]  # September Experiment
N2 = [0.26, 0.30, 0.36, 0.39, 0.39, 0.41]  # November Experiment

In [4]:
# Function to extract LIF value from parameters
def lif(parameters):
    return float(parameters.split("LIF")[1].split("A")[0])

# Function to extract A value from parameters
def ave(parameters):
    return float(parameters.split("A")[1].split("C")[0])

# Function to extract C value from parameters
def com(parameters):
    return float(parameters.split("C")[1].split("B")[0])

# Function to extract B value from parameters
def bef(parameters):
    return float(parameters.split("B")[1].split("MTD")[0])

# Function to extract MTD value from parameters
def mtd(parameters):
    return float(parameters.split("B")[1])  # This seems to get the MTD value directly

# Function to calculate the maximum deviation for March experiment
def mar(f, e, d, c, b, a):
    return np.round(np.max([np.abs(M2[5]-f), np.abs(M2[4]-e),
                             np.abs(M2[3]-d), np.abs(M2[2]-c),
                             np.abs(M2[1]-b), np.abs(M2[0]-a)]), 2)

# Function to calculate the maximum deviation for September experiment
def sep(f, e, d, c, b, a):
    return np.round(np.max([np.abs(S2[5]-f), np.abs(S2[4]-e),
                             np.abs(S2[3]-d), np.abs(S2[2]-c),
                             np.abs(S2[1]-b), np.abs(S2[0]-a)]), 2)

# Function to calculate the maximum deviation for November experiment
def nov(f, e, d, c, b, a):
    return np.round(np.max([np.abs(N2[5]-f), np.abs(N2[4]-e),
                             np.abs(N2[3]-d), np.abs(N2[2]-c),
                             np.abs(N2[1]-b), np.abs(N2[0]-a)]), 2)

In [5]:
# Apply the extraction functions to the DataFrame
nf['LIF'] = nf.apply(lambda row: lif(row[-1]), axis=1)
nf['A'] = nf.apply(lambda row: ave(row[-1]), axis=1)
nf['C'] = nf.apply(lambda row: com(row[-1]), axis=1)
nf['B'] = nf.apply(lambda row: bef(row[-1]), axis=1)
nf['MTD'] = nf.apply(lambda row: mtd(row[-1]), axis=1)

# Calculate maximum deviations for each experiment
nf['Mar15'] = nf.apply(lambda row: mar(row[90], row[75], row[60], row[45], row[30], row[15]), axis=1)
nf['Sep15'] = nf.apply(lambda row: sep(row[90], row[75], row[60], row[45], row[30], row[15]), axis=1)
nf['Nov15'] = nf.apply(lambda row: nov(row[90], row[75], row[60], row[45], row[30], row[15]), axis=1)


In [6]:
# Filter the DataFrame for each experiment based on the maximum deviation
MAR = nf.sort_values('Mar15')
MAR = MAR[MAR['Mar15'] < 0.05].reset_index(drop=True)  # Filter and reset index

SEP = nf.sort_values('Sep15')
SEP = SEP[SEP['Sep15'] < 0.05].reset_index(drop=True)  # Filter and reset index

NOV = nf.sort_values('Nov15')
NOV = NOV[NOV['Nov15'] < 0.05].reset_index(drop=True)  # Filter and reset index